In [121]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from pathlib import Path
from dotenv import load_dotenv
from config import settings
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss
from langchain_text_splitters import RecursiveCharacterTextSplitter
load_dotenv()

db_path = Path("db")
db_path.mkdir(exist_ok=True)


embeddings = OpenAIEmbeddings(openai_api_key=settings.OPENAI_API_KEY, openai_proxy=settings.OPENAI_PROXY,model=settings.MODEL_EMBEDDING_NAME)
index_db = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))


vector_store = FAISS(
        embedding_function=embeddings,
        index=index_db,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        )
    
vector_store.save_local(str(db_path))


text_splitter_1200_600 = RecursiveCharacterTextSplitter(
                    chunk_size=1200, chunk_overlap=600)


In [143]:
db_current_work_noncode = FAISS.load_local("db/", embeddings, allow_dangerous_deserialization=True)
db_current_work_noncode.docstore._dict

{}

In [154]:
len(db_current_work_noncode.docstore._dict)

28

In [ ]:
db_current_work_noncode.docstore._dict

In [152]:
db_current_work_noncode.save_local("db/")

In [145]:
import bs4
import ssl
import asyncio
import nest_asyncio
from langchain_community.document_loaders import WebBaseLoader

# Включаем поддержку вложенных циклов событий
nest_asyncio.apply()

# Отключаем проверку SSL сертификата
ssl._create_default_https_context = ssl._create_unverified_context


urls = [
    "https://help.konsol.pro/quick-registration#block-8b788d59acb4437da171d1c9eac2832d",
    "https://help.konsol.pro/registration-smz", 
    "https://help.konsol.pro/registration-gph",
    "https://help.konsol.pro/registration-ip",
    "https://help.konsol.pro/pin-code",
    "https://help.konsol.pro/download-application",
    "https://help.konsol.pro/signin-in-app",
    "https://help.konsol.pro/signin-from-pc",
    "https://help.konsol.pro/8caf63ab852249b5a691d36ed73960fe",
    "https://help.konsol.pro/telegram-bot",
    "https://help.konsol.pro/telegram-miniapp"
]


loader = WebBaseLoader(
    web_paths=urls,
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(class_="notion-root max-width has-footer"),
    },
    bs_get_text_kwargs={"separator": "  ", "strip": True},
    #bs_get_text_kwargs={"strip": True},
    verify_ssl=False,
    continue_on_failure=True
)

headers_loader = WebBaseLoader(
    web_paths=urls,
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(class_="notion-header__title"),
    },
    bs_get_text_kwargs={"separator": "  ", "strip": True},
    verify_ssl=False,
    continue_on_failure=True
)


docs = []
headers = []

async def load_docs():
    async for doc in loader.alazy_load():
        docs.append(doc)
    print(f"Загружено документов: {len(docs)}")


async def load_docs_headers():
    async for header in headers_loader.alazy_load():
        headers.append(header)
    print(f"Загружено документов: {len(headers)}")

asyncio.run(load_docs())
asyncio.run(load_docs_headers())

for doc in docs:
    doc.metadata["header"] = headers[docs.index(doc)].page_content
    doc.metadata["table_of_contents"] = "chunk_1200_600"



Fetching pages: 100%|##########| 11/11 [00:02<00:00,  3.88it/s]


Загружено документов: 11


Fetching pages: 100%|##########| 11/11 [00:02<00:00,  4.91it/s]

Загружено документов: 11


In [146]:
print(docs[0])

page_content='Что это  Преимущества регистрации через Сбер ID/Тинькоф ID  Как это работает  Какие данные может получить Консоль от банка при вашей регистрации через Тинькофф/Сбер ID при их наличии  Что это  Сбер/Тинькофф ID это функции регистрации на платформе Консоль, позволяющие авторизоваться с помощью существующих данных пользователя.  Преимущества регистрации через Сбер ID/Тинькоф ID  Вы можете пройти регистрацию за три минуты, за счет импорта и предзаполнения данных.  Авторизация по Сбер ID  ✅  Быстрая регистрация.  ✅ Не нужно грузить сканы документов, если они не запрашиваются компанией отдельно.  ✅ Не нужно грузить селфи.  Авторизация по Тинькофф ID  ✅ Быстрая регистрация.  ✅ Не нужно грузить сканы документов, если они не запрашиваются компанией отдельно.  ✅ Не нужно грузить селфи.  ✅ Не нужно вводить реквизиты счета.  Как это работает  Вам придет SMS приглашение от сотрудничающей с вами компании.  Пройдите по ссылке и выберите  «‎Войти с Тинькофф ID»/«‎Войти по Сбер ID»;  Посл

In [147]:
print(docs[0].metadata)

{'source': 'https://help.konsol.pro/quick-registration#block-8b788d59acb4437da171d1c9eac2832d', 'header': 'Самая быстрая регистрация на платформе по Сбер/Тинькоф ID', 'table_of_contents': 'chunk_1200_600'}


In [148]:
import re

def preprocess_text(text: str) -> str:

    text = re.sub(r'[^а-яА-ЯёЁa-zA-Z0-9\s]', ' ', text)
    
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

processed_texts = [preprocess_text(doc.page_content) for doc in docs]

for doc in docs:
    doc.page_content = processed_texts[docs.index(doc)]


In [149]:
from langchain_core.documents import Document
def doc_chunks_1200_600(content, url, header_or_text):
    return  Document(
        page_content=content,
        metadata={"table": "chunks_1200_600",
                "url": url,
                "header": header_or_text},
    )

In [150]:
all_splits = text_splitter_1200_600.split_text(docs[0].page_content)
all_splits

['Что это Преимущества регистрации через Сбер ID Тинькоф ID Как это работает Какие данные может получить Консоль от банка при вашей регистрации через Тинькофф Сбер ID при их наличии Что это Сбер Тинькофф ID это функции регистрации на платформе Консоль позволяющие авторизоваться с помощью существующих данных пользователя Преимущества регистрации через Сбер ID Тинькоф ID Вы можете пройти регистрацию за три минуты за счет импорта и предзаполнения данных Авторизация по Сбер ID Быстрая регистрация Не нужно грузить сканы документов если они не запрашиваются компанией отдельно Не нужно грузить селфи Авторизация по Тинькофф ID Быстрая регистрация Не нужно грузить сканы документов если они не запрашиваются компанией отдельно Не нужно грузить селфи Не нужно вводить реквизиты счета Как это работает Вам придет SMS приглашение от сотрудничающей с вами компании Пройдите по ссылке и выберите Войти с Тинькофф ID Войти по Сбер ID После этого вы будете перенаправлены на страницу Тинькофф или Сбер где ну

In [151]:
from uuid import uuid4
s4 = 0
for doc in docs:

    all_splits = text_splitter_1200_600.split_text(doc.page_content)
    doc_list = []

    for split in all_splits:
        doc_list.append(doc_chunks_1200_600(split, doc.metadata["source"], doc.metadata["header"]))
    

    uuids = [str(uuid4()) for _ in range(len(doc_list))]
    db_current_work_noncode.add_documents(documents=doc_list, ids=uuids)

    s4+=1
    print("-----------",s4,"-----------")


----------- 1 -----------
----------- 2 -----------
----------- 3 -----------
----------- 4 -----------
----------- 5 -----------
----------- 6 -----------
----------- 7 -----------
----------- 8 -----------
----------- 9 -----------
----------- 10 -----------
----------- 11 -----------


In [131]:
#Евклидка с фильтром
results = db_current_work_noncode.similarity_search(
    "Как открыть статус самозанятого",
    k=10,
    filter={"table": "chunks_1200_600"},
)
for current_results in results:
    print(current_results.metadata)

{'table': 'chunks_1200_600', 'url': 'https://help.konsol.pro/registration-smz', 'header': 'Регистрация для самозанятых'}
{'table': 'chunks_1200_600', 'url': 'https://help.konsol.pro/quick-registration#block-8b788d59acb4437da171d1c9eac2832d', 'header': 'Самая быстрая регистрация на платформе по Сбер/Тинькоф ID'}
{'table': 'chunks_1200_600', 'url': 'https://help.konsol.pro/registration-smz', 'header': 'Регистрация для самозанятых'}
{'table': 'chunks_1200_600', 'url': 'https://help.konsol.pro/registration-ip', 'header': 'Регистрация для ИП'}
{'table': 'chunks_1200_600', 'url': 'https://help.konsol.pro/registration-smz', 'header': 'Регистрация для самозанятых'}
{'table': 'chunks_1200_600', 'url': 'https://help.konsol.pro/registration-smz', 'header': 'Регистрация для самозанятых'}
{'table': 'chunks_1200_600', 'url': 'https://help.konsol.pro/quick-registration#block-8b788d59acb4437da171d1c9eac2832d', 'header': 'Самая быстрая регистрация на платформе по Сбер/Тинькоф ID'}
{'table': 'chunks_120

In [139]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "Как открыть статус самозанятого"

llm = ChatOpenAI(temperature=0,
                 model=settings.MODEL_NAME,
                 openai_api_key=settings.OPENAI_API_KEY)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db_current_work_noncode.as_retriever(search_kwargs = {"k" : 10, 'filter': {'table':'chunks_1200_600'}}), llm=llm
)
unique_docs = retriever_from_llm.invoke(question)

In [140]:
unique_docs

[Document(id='d76fa23f-f2d1-46ce-9117-f023226e030e', metadata={'table': 'chunks_1200_600', 'url': 'https://help.konsol.pro/quick-registration#block-8b788d59acb4437da171d1c9eac2832d', 'header': 'Самая быстрая регистрация на платформе по Сбер/Тинькоф ID'}, page_content='вы уже являетесь самозанятым нажмите Я самозанятый Если вы еще не являетесь самозанятым нажмите кнопку Хочу стать самозанятым и следуйте одной из инструкций Стать самозанятым через Сбербанк Онлайн Стать самозанятым через Мой налог с паспортом Стать самозанятым через Мой налог при помощи аккаунта Госуслуги Если были получены данные от банка на следующем шаге вы увидите предзаполненную анкету и или реквизиты счета Пример предзаполненой анкеты при регистрации через Сбер Если из банка были получены данные паспорта регистрации или СНИЛС то вам не нужно будет загружать сканы этих документов Пример предзаполненных данных паспорта при регистрации через Сбер Какие данные может получить Консоль от банка при вашей регистрации через 

In [138]:
from langchain_core.runnables.config import RunnableConfig
from langgraph_app.chat_graph import rag_graph


question = "Как открыть статус самозанятого"
message_state = {"messages": [{"role": "user", "content": question}], "context": ""}

config = RunnableConfig(
    configurable={
        "api_key": settings.OPENAI_API_KEY,
        "proxy": settings.OPENAI_PROXY,
        "tokens": None,
        "temperature": None,
        "model_name_gpt": settings.MODEL_NAME,
        "model_name_embedding": settings.MODEL_EMBEDDING_NAME,
        "path_to_vector_store": "db/"
    }
)

ai_response_text = await rag_graph.ainvoke(message_state, config=config)
ai_response_text

KeyError: 'messages'